## SILVER TO GOLD LAYER

### Gold Layer - Profit and Loss

In [1]:
# Importing Common Utility Funcation

import pandas as pd
from ETLTools import GlobalPath, Portfolio

In [2]:
# Import necessary libraries and utility functions
%run ../COMMON/common_utility.ipynb

In [3]:
# Instantiate GlobalPath
tradehistory_silver_file_path = GlobalPath(
    "DATA/SILVER/TradeHistory/TradeHistory_data.csv"
)
profitloss_gold_file_path = GlobalPath(
    "DATA/GOLD/ProfitLoss/ProfitLoss_data.csv"
)

### Data Processing

- Read and sort trade history data.
- Apply portfolio trade logic.


In [4]:
# Read the CSV file
df_trade_history = pd.read_csv(tradehistory_silver_file_path)

# Convert 'datetime' to datetime type
df_trade_history["datetime"] = pd.to_datetime(df_trade_history["datetime"])

# Sort the DataFrame by 'datetime'
df_trade_history = df_trade_history.sort_values(by="datetime")

print(
    f"Read SILVER Layer trade history data from: {tradehistory_silver_file_path}"
)

Read SILVER Layer trade history data from: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\SILVER\TradeHistory\TradeHistory_data.csv


### Portfolio Logic Application

- Instantiate Portfolio and apply trade logic.
- Handle expired stocks


In [5]:
# Apply the trade logic to each row of the DataFrame
# Instantiate the Portfolio object
portfolio = Portfolio()
for record in df_trade_history.to_dict(orient="records"):
    portfolio.trade(record)

portfolio.check_expired_stocks()
# expired_stocks example : df_trade_history["scrip_name"] == "NIFTY-PE-24650-18JUL2024"

NIFTY-PE-21650-04JAN2024 => 50 expired
NIFTY-PE-24400-25JUL2024 => 25 expired


### Final Processing and Export

- Select and sort relevant columns.
- Save the processed data as a CSV file in the Gold layer.


In [6]:
# Create a DataFrame from the processed data
df_pnl = pd.DataFrame(portfolio.get_pnl())

# Sort the DataFrame
df_pnl = df_pnl.sort_values(
    by=[
        "segment",
        "exchange",
        "symbol",
        "scrip_name",
        "close_datetime",
        "open_datetime",
    ]
)

# Round the values in to two decimal places
df_pnl = df_pnl.round(2)

# Reset index to ensure it starts from 0
df_pnl = df_pnl.reset_index(drop=True)

# Select and reorder the columns for the final DataFrame
df_pnl = df_pnl[
    [
        "exchange",
        "segment",
        "symbol",
        "scrip_name",
        "position",
        "quantity",
        "open_datetime",
        "open_side",
        "open_price",
        "open_amount",
        "close_datetime",
        "close_side",
        "close_price",
        "close_amount",
        "pnl_amount",
        "pnl_percentage",
    ]
]

# Save the final DataFrame to a CSV file
df_pnl.to_csv(profitloss_gold_file_path, index=None)

print("GOLD Layer CSV file for ProfitLoss successfully created at:")
print(profitloss_gold_file_path)

# Display the DataFrame debugrmation
df_pnl.info()

GOLD Layer CSV file for ProfitLoss successfully created at:
D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\GOLD\ProfitLoss\ProfitLoss_data.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231 entries, 0 to 230
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   exchange        231 non-null    object        
 1   segment         231 non-null    object        
 2   symbol          231 non-null    object        
 3   scrip_name      231 non-null    object        
 4   position        231 non-null    object        
 5   quantity        231 non-null    int64         
 6   open_datetime   231 non-null    datetime64[ns]
 7   open_side       231 non-null    object        
 8   open_price      231 non-null    float64       
 9   open_amount     231 non-null    float64       
 10  close_datetime  231 non-null    datetime64[ns]
 11  close_side      231 non-null    object        